In [2]:
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
from selenium.webdriver.common.by import By
import pandas as pd
import tqdm
import pickle as pkl

## Minerador de URL's

In [11]:
## Escolhar o site com o esporte selecionado:
url = "https://www.tsn.ca/tennis"

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

driver.set_window_size(1920, 1080)

driver.get(url)

time.sleep(1)

## Looping infinito para a coleta dos dados:
while True:

    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight)")

    time.sleep(1)

    driver.find_element(By.XPATH, '//button[normalize-space()="Load More"]').click()

    time.sleep(3)

    links = driver.find_elements(By.CLASS_NAME, "blog-post-vert")

    links_suit = [x.find_element(By.TAG_NAME, "a").get_attribute("href") for x in links]

    time.sleep(2)

    print(len(links_suit))

    ## Backup da lista de links
    with open(f"Backup_{url.split('/')[-1]}.pkl", "wb") as f:
        pkl.dump(links_suit, f)

driver.close()

26
36
46
56
66
76
86
96
106
116
126
136
146
156
166
176
186
196
206
216
226
236
246
256
266
276
286
296
306
316
326
336
346
356
366
376
386
396
406
416
426
436
446
456
466
476
486
496
506
516
526
526
546
556
566
576
586
596
606
616
626
636
646
656
666
676
686
696
706
716
726
736
746
756
766
776
786
796
806
816
826
836
846
856
866
876
886
896
906
916
926
936
946
956
966
976
986
996
1006
1016
1026
1026
1046
1056
1066
1076
1086
1096
1106
1116
1126
1136
1146
1156
1166
1176
1186
1196
1206
1216
1226
1236
1246
1256
1266
1276
1286
1296
1306
1316
1326
1336
1346
1356
1366
1376
1386
1396
1406
1416
1426
1436
1446
1456
1466
1476
1486
1496
1506
1516
1526
1536
1546
1556
1566
1576
1586
1596
1606
1616
1626
1636
1646
1656
1666
1676
1686
1696
1706
1716
1726
1736
1746
1756
1766
1776
1786
1796
1806
1816
1826
1836
1846
1856
1866
1876
1886
1896
1906
1916
1926
1936
1946
1956
1966
1976
1986
1996
2006
2016
2026
2036
2046
2056
2066
2076
2086
2096
2106
2116
2126
2136
2146
2156
2166
2176
2186
2196
2206
2216
2226
2

KeyboardInterrupt: 

In [12]:
with open("nba_links_news2.pkl", "rb") as f:
    links = pkl.load(f)

In [13]:
links = set(links)

9054

In [24]:
df = pd.read_csv("nba_news.csv")

for link in tqdm.tqdm(links[5138:], "News Count"):
    response = requests.get(link)

    html = response.content

    try:

        soup = BeautifulSoup(html, "html.parser")

        title = soup.find("h1", class_="ArticleHeader_ahTitle__mIo92").text

        author = soup.find("p", class_="ArticleAuthor_authorName___AnQD").text

        data = soup.find("time", class_="ArticleHeader_ahDate__J3fwr").text

        content = soup.find("div", class_="ArticleContent_article__NBhQ8")
        content = content.find_all("p")
        content = [x.text for x in content] 
        content = " ".join([x for x in content])

        df.loc[df.shape[0]] = [title, author, data, content]
        df.to_csv("nba_news.csv", index=False)

    except:
        continue

News Count: 100%|██████████| 1178/1178 [28:29<00:00,  1.45s/it]


In [25]:
df = pd.read_csv("nba_news.csv")
df.shape

(6262, 4)

In [84]:
for link in links:
    response = requests.get(link)

    html = response.content

    soup = BeautifulSoup(html, "html.parser")

    pre_links = soup.find_all("div", class_="css-175oi2r r-1lwhwsw r-1wzrnnt r-d9fdf6 r-1knelpx r-13qz1uu")

    if len(pre_links) > 1:
        links = list(set([x["href"] for x in pre_links[1].find_all("a") if "stories" in x["href"]]))
    else:
        links = list(set([x["href"] for x in pre_links[0].find_all("a") if "stories" in x["href"]]))

    for link in links:
        url_site = "https://www.basketballnews.com" + link
        site = requests.get(url_site)

        html_site = site.content
        site_soup = BeautifulSoup(html_site, "html.parser")

        title = site_soup.find("h1").text
        author = site_soup.find("div", class_="css-1rynq56 r-1niwhzg r-t1lq8t r-1231a37 r-1enofrn r-356f0p r-tsynxw r-1xnzce8").text
        data = site_soup.find("div", class_="css-1rynq56 r-1niwhzg r-t1lq8t r-1231a37 r-1enofrn r-356f0p r-1xnzce8").text
        body = [x.text for x in site_soup.find("div", {"itemprop":"articleBody"}).find_all("p")]
        body = " ".join(body)

        news.loc[news.shape[0]] = [title, author, data, body]

    news.to_csv("news.csv", index=False)
    print(f"Page {i} was crawled.")


['https://www.nba.com/news/qa-rick-carlisle-pacers-coach-outlook',
 'https://www.nba.com/news/nba-fantasy-implications-of-offseason-coaching-changes',
 'https://www.nba.com/news/qa-taylor-jenkins-grizzlies-2023',
 'https://www.nba.com/news/this-date-in-nba-history-2022-23',
 'https://www.nba.com/news/30-teams-30-days-raptors-2023-24',
 'https://www.nba.com/news/30-teams-30-days-wizards-2023-24',
 'https://www.nba.com/news/30-teams-30-days-trailblazers-2023-24',
 'https://www.nba.com/news/victor-wembanyama-reaffirms-olympic-commitment',
 'https://www.nba.com/news/celtics-sign-neemias-queta-to-2-way-contract',
 'https://www.nba.com/news/nba-fantasy-implications-of-free-agency-trades-for-2023-24',
 'https://www.nba.com/news/30-teams-30-days-hawks-2023-24',
 'https://www.nba.com/news/report-kelly-oubre-jr-expected-to-sign-with-philadelphia-76ers',
 'https://www.nba.com/news/foundation-friday-spotlighting-excellence-in-education',
 'https://www.nba.com/news/jarred-vanderbilt-extension-laker